# 時間序列資料分析
## 以台灣再生能源發電佔比為例

本專案使用由經濟部能源署公開的國家發電量資料，統計過去 20 年 (2006/01-2025/12) 逐月之全國總用電量及各類別的總發電量。配合 2023 年政府原先訂定預計 2025 年達 20% 再生能源發電佔比的目標，研究欲以再生能源發電佔比時間序列作為反應變數，觀察序列變化趨勢與季節波動，並建立與挑選合適的時間序列模型，以解釋過去趨勢及預測近期政府公布預計推遲達標時間點：2026 年 10 月之佔比。

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the dataset
path = "../data/power_generation_11412.csv"
df = pd.read_csv(path)
df['Period'] = pd.to_datetime(df['Period'].astype(str), format='%Y%m')
df = df.sort_values('Period').set_index('Period').asfreq('MS')
print(df.index)

renewable_cols = [
    'Solar_PV', 'Wind_Onshore', 'Wind_Offshore',
    'Biomass_Solid', 'Biomass_Biogas',
    'Waste', 'Hydro', 'Geothermal'
]
all_sources = [
    'Pumped_Storage','Nuclear',
    'Solar_PV', 'Wind_Onshore', 'Wind_Offshore',
    'Biomass_Solid', 'Biomass_Biogas',
    'Waste', 'Hydro', 'Geothermal',
    'Coal_Fired','Oil_Fired','LNG_Fired'
]

df = df.assign( 
    Renewable_Generation=lambda x: x[renewable_cols].sum(axis=1), 
    Total_Generation=lambda x: x[all_sources].sum(axis=1), 
    Renewable_Share=lambda x: x['Renewable_Generation'] / x['Total_Generation']
)

y_t = df['Renewable_Share']
y_train = df['Renewable_Share'][:-2]
y_test = df['Renewable_Share'][-2:]

x_t = df['exog'] = np.where(df.index >= '2021-04-01', 1, 0)
x_train = df['exog'][:-2]
x_test = df['exog'][-2:]

本研究關注之反應變數 $𝑦_t$ -- 再生能源發電佔比，其定義如下：

$$
\text{再生能源發電佔比}_t
=
\frac{\text{再生能源發電量}_t}{\text{總發電量}_t}
$$

其中 $t$ 表示時間。

In [ ]:
y_t.plot(figsize=(10, 4))
plt.title('Monthly Renewable Energy Share')
plt.ylabel('Share')
plt.show()

在進行時序分析前，首先將反應變數 $\text{再生能源發電佔比}_t$ 進行視覺化，以觀察時間序列整體走勢變化與可能的結構特徵。

由上圖可觀察到：
1. 再生能源占比整體呈現長期上升趨勢
2. 資料存在季節性波動
3. 序列在後段波動幅度明顯變大

上述現象均顯示該時間序列並非**平穩(stationary)**。透過 STL(Seasonal-Trend decomposition using LOESS) 將原序列拆分為趨勢 (Trend)、季節性變化 (Seasonal) 與剩餘的殘差 (Residuals) 部分，可以更進一步地將肉眼對序列變化的觀察量化出來。

In [ ]:
from statsmodels.tsa.seasonal import STL
stl = STL(y_t, period=12, robust=True)
stl.fit().plot()
plt.show()

STL 分解結果顯示：
1. 趨勢項呈現明顯且非線性的上升趨勢，其中約在 2021 年中有明顯轉折點。
2. 季節性成分穩定存在，後段波動幅度較大。
3. 殘差大致圍繞 0 波動，後段變異亦增加，呈開口向右喇叭狀分布。

綜上，時間序列的分解結果顯示資料同時具有明顯的趨勢與季節性結構，支持後續採用能同時刻畫趨勢與季節效應之時間序列模型進行分析。本研究依循 Box–Jenkins 方法，首先透過差分使原始時間序列趨於平穩；接著利用差分後序列之 ACF 與 PACF，判斷非季節/季節部分之自迴歸 (AR) 與移動平均 (MA) 項適當階數；而後進一步將外生變數 (X) 納入模型中，以評估其是否能有效捕捉時間序列中潛在的結構性轉折，並提升模型對趨勢變化之解釋能力；最終以 GARCH 模型擬合 SARIMAX 模型殘差部分，修正資料不滿足 SARIMAX 模型假設變異數相等的行為，使模型也能考慮到波動幅度逐漸增大的變化。

In [ ]:
y_diff = y_t.diff()
y_diff.plot(figsize=(10,4))
plt.title('First Difference of Renewable Share')
plt.show()

針對趨勢結構，由於趨勢並未呈現明顯的大幅成長衰退或 S 型轉折，因此考慮對原始時間序列做一階差分 (d=1) 來處理。由上圖對一階差分序列的視覺化可觀察到，大致上新序列已消除趨勢的變化，圍繞著 0 的均線上下震盪，接近弱平穩的狀態。從以下對一階差分的 ACF 圖中我們可以發現到 $\text{lag}=1$ 出現負訊號，之後呈現接近截斷狀態，可以初步判斷模型的 MA 部分參數 q=1；而 $\text{lag}=6,8,12,24$ 出現部分較強訊號，得進一步考慮季節模型 (SARIMA) 來吸收季節性變動。至於 PACF 圖中，訊號沒有連續結構、偶有零星訊號、沒有截尾趨勢，因此接下來模型中的 AR 部分會透過擬合不同階數之模型 (p=0,1,2)，比較各模型之表現來決定參數 p。

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(y_diff.dropna(), lags=48)
plot_pacf(y_diff.dropna(), lags=48)
plt.show()

綜合上述觀點，由 $ARIMA(0,1,1)$ 模型出發，來觀察看看模型是否有捕捉到時間序列的趨勢。第一張 $ARIMA(0,1,1)$ 模型的殘差 ACF 圖中可觀察到，$\text{lag}=6,8,18,24$ 時仍有大於區間的值，並且 Ljung-Box test 之$\text{p-value} < 0.001$，表示檢定拒絕這個模型的殘差為白噪音，仍有重要趨勢未被模型掌握。基於這個結果，嘗試調整模型 AR 部分之參數 p=1 或 2 來觀察模型的表現。

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
# ARIMA(0,1,1) model
p = 0
q = 1
mod_df = p+q
nlag = 24
mod_011 = ARIMA(
    y_t, 
    order=(p,1,q)
).fit()

print(mod_011.summary())

lb = acorr_ljungbox(
    mod_011.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_011.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()

將前述 $ARIMA(0,1,1)$ 和以下 $ARIMA(1,1,1)$、$ARIMA(2,1,1)$ 模型做比較，經由模型的 AIC 和 BIC 評估，$ARIMA(1,1,1)$ 兩個指標皆為最小，表現為三者中最優。而從模型參數的假說檢定可以發現到二階 AR 的係數信賴區間包含 0，無法拒絕參數為 0 的虛無假設，顯示 p=2 即出現參數冗餘的情形；然而，透過 Ljung-Box檢定，即便擬合較佳的 $ARIMA(1,1,1)$ 模型，檢定結果 p-value 仍小於 0.01，拒絕其殘差部分為白噪音的虛無假設，代表仍有重要趨勢未考慮進模型中。最初對於原始時間序列的的 STL 分解中，便可觀察到序列的季節項分解有明顯的周期波動，因此接著將更進一步考慮納入季節項，擴展為 SARIMA 模型來捕捉週期性的變化。

In [ ]:
# ARIMA(1,1,1) model
p = 1
q = 1
mod_df = p+q
nlag = 24
mod_111 = ARIMA(
    y_t, 
    order=(p,1,q)
).fit()

print(mod_111.summary())

lb = acorr_ljungbox(
    mod_111.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_111.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()

In [ ]:
# ARIMA(2,1,1) model
p = 2
q = 1
mod_df = p+q
nlag = 24
mod_211 = ARIMA(
    y_t, 
    order=(p,1,q)
).fit()

print(mod_211.summary())

lb = acorr_ljungbox(
    mod_211.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_211.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()

對於月資料，嘗試對一階差分後的時間序列再進行 $\text{lag}=12$ 的季節性差分來捕捉殘留的季節性。對經過非季節差分與季節差分的新序列做 ACF/PACF 圖，可以從下兩張圖觀察到 $\text{lag}=12$ 的部分皆出現了明顯的負相關，為明顯的過度差分訊號。因此，接下來將以 $ARIMA(1,1,1)$ 模型作為比較基準，在模型中固定季節差分參數 D=0，透過調整季節性 AR/MA 來處理剩餘未捕捉的波動。以下將比較三個模型:
1. 僅增加季節MA的 $SARIMA(1,1,1)(0,0,1)_{12}$
2. 僅增加季節AR的 $SARIMA(1,1,1)(1,0,0)_{12}$
3. 同時考慮季節AR/MA的 $SARIMA(1,1,1)(1,0,1)_{12}$
觀察模型殘差的ACF圖有無趨勢、以 Ljung-Box test 檢定模型殘差，並且比較三模型的 AIC/BIC 作為選模的依據。

In [ ]:
y_seasonal = y_t.diff().diff(12)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

gs = axes[0, 0].get_gridspec()
for ax in axes[0, :]:
    ax.remove()
ax_ts = fig.add_subplot(gs[0, :])

y_seasonal.plot(ax=ax_ts, color='tab:blue')
ax_ts.set_title('Differenced Renewable Share (d=1, D=1, s=12)', fontsize=14)
ax_ts.grid(True, alpha=0.3)

ax_acf = axes[1, 0]
plot_acf(y_seasonal.dropna(), lags=48, ax=ax_acf)
ax_acf.set_title('Autocorrelation (ACF)')

ax_pacf = axes[1, 1]
plot_pacf(y_seasonal.dropna(), lags=48, ax=ax_pacf)
ax_pacf.set_title('Partial Autocorrelation (PACF)')

plt.tight_layout()
plt.show()

從ACF圖觀察，模型僅加入季節 MA 項與僅加入季節 AR 項，和 $ARIMA(1,1,1)$ 模型的圖形趨勢仍然接近，儘管從殘差檢定和 AIC/BIC 等指標來看模型接略有改善。僅是納入季節AR項，Ljung-Box test 就已達到不顯著($\text{p-value}=0.0774>0.05$)，在 95% 信心水準下無法拒絕虛無假設：模型殘差為白噪音，說明這個模型已達可接受的程度。然而，同時考慮季節MA與季節AR項的模型：$SARIMA(1,1,1)(1,0,1)_{12}$，其殘差 ACF 圖之 lag 訊號更為集中，並且 Ljung-Box test 之 p-value 達 0.2281 高於 $SARIMA(1,1,1)(1,0,0)_{12}$ 模型、AIC/BIC 皆低於前者，各項指標皆顯示 $SARIMA(1,1,1)(1,0,1)_{12}$ 模型擬合更佳。

In [ ]:
# SARIMA(1,1,1)(0,0,1)_12 model
p = 1
q = 1
P = 0
Q = 1
mod_df = p+q+P+Q
nlag = 24
mod_111001 = SARIMAX(
    y_t, 
    order=(p,1,q), 
    seasonal_order=(P,0,Q,12)
).fit()

print(mod_111001.summary())
lb = acorr_ljungbox(
    mod_111001.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_111001.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()

In [ ]:
# SARIMA(1,1,1)(1,0,0)_12 model
p = 1
q = 1
P = 1
Q = 0
mod_df = p+q+P+Q
nlag = 24
mod_111100 = SARIMAX(
    y_t, 
    order=(p,1,q), 
    seasonal_order=(P,0,Q,12)
).fit()

print(mod_111100.summary())
lb = acorr_ljungbox(
    mod_111100.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_111100.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()

In [ ]:
# SARIMA(1,1,1)(1,0,1)_12 model
p = 1
q = 1
P = 1
Q = 1
mod_df = p+q+P+Q
nlag = 24
mod_111101 = SARIMAX(
    y_t, 
    order=(p,1,q), 
    seasonal_order=(P,0,Q,12)
).fit()

print(mod_111101.summary())
lb = acorr_ljungbox(
    mod_111101.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_111101.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()

進一步地，本研究將趨勢轉折之解釋變數納入模型，建立 $SARIMAX(1,1,1)(1,0,1)_{12}$ 以評估外生資訊:宣示 2050 淨零碳排對再生能源占比變動之解釋效果。由估計結果可見，解釋變數 $x_t$ 並未達統計顯著（$\text{p-value}=0.899$），顯示在控制時間序列自身的自相關與季節結構後，該轉折指標對再生能源占比之邊際影響有限，資料中的趨勢變化主要仍由時間序列內生動態所刻畫。

然而，就整體模型配適而言，納入外生變數後之模型 AIC = -1630.25、BIC = -1609.39，相較未納入解釋變數之 $SARIMA(1,1,1)(1,0,1)_{12}$ 模型皆下降，並且使得原先無法拒絕其為 0 之季節 MA 項係數達統計顯著，顯示納入解釋變數改善時間序列模型結構之穩定性。從殘差診斷觀察，Ljung–Box 檢定於 lag 24 之 $\text{p-value} = 0.3200 > 0.05$，在 95% 信心水準下無法拒絕「殘差為白噪音」之虛無假設，說明模型已充分擷取資料之時間相依與季節性結構，殘差近似隨機波動，模型配適屬良好水準。

綜合係數顯著性、AIC/BIC 準則與殘差檢定結果，雖然外生解釋變數未呈現統計顯著影響，但模型整體表現維持穩定且配適略有提升，顯示再生能源占比之長期變動主要仍受時間序列內生趨勢與季節結構主導。基於模型具良好白噪音殘差性質與最低 AIC/BIC 值，後續分析採用此 $SARIMAX(1,1,1)(1,0,1)_{12}$ 模型作為推論與預測之基礎。

In [ ]:
# SARIMAX(1,1,1)(1,0,1)_12 model with exogenous variable
p = 1
q = 1
P = 1
Q = 1
mod_df = p+q+P+Q
nlag = 24

mod_sarimax = SARIMAX(
    y_t,
    order=(1,1,1),
    seasonal_order=(1,0,1,12),
    exog = x_t,
).fit()

print(mod_sarimax.summary())
lb = acorr_ljungbox(
    mod_sarimax.resid.dropna(),
    lags=[nlag],
    model_df=mod_df,
    return_df=True
).assign(resid_df = nlag-mod_df)
print(lb)

mod_sarimax.plot_diagnostics(figsize=(12,10), lags=24)
plt.show()


接著由殘差診斷圖可觀察到，殘差的波動幅度隨時間推進逐漸擴大，呈現向右開口的喇叭狀型態，顯示殘差可能存在**變異數不恆定（heteroskedasticity）**的現象。為進一步檢驗此問題，對模型殘差進行 Engle's ARCH LM 檢定，其結果如下：

$$
\text{Lagrange Multiplier (LM)} = 35.4934\text{, p-value} < 0.001
$$

由於 p-value 顯著小於 0.05，拒絕虛無假設：「無 ARCH 效應」，表示殘差中存在顯著的變異數不恆定現象，亦即殘差變異數隨時間變動，違反 ARIMA/SARIMA 模型同質變異數之假設。此結果說明目前模型雖能捕捉平均結構（mean equation），但未能充分描述波動結構（volatility dynamics），後續考慮建立 SARIMA + GARCH 模型，以兩階段的模型分別預測條件平均數與條件變異數來做最後的統計推論。

In [ ]:
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model

sarimax_std_resid = mod_sarimax.filter_results.standardized_forecasts_error[0]
sarimax_std_resid = pd.Series(sarimax_std_resid).dropna()
lm_stat, p_value, f_stat, f_p_value = het_arch(sarimax_std_resid)

print("Engle's ARCH Test (LM Test) Results")
print(f"{'Statistic Type':<25} | {'Value':<12} | {'p-value':<12}")
print("-" * 50)
print(f"{'Lagrange Multiplier (LM)':<25} | {lm_stat:<12.4f} | {p_value:<12.4f}")
print(f"{'F-statistic':<25} | {f_stat:<12.4f} | {f_p_value:<12.4f}")

在條件平均數部分，SARIMAX 模型在殘差自相關檢定下大致已無顯著序列相關，但由前述 Engle’s ARCH LM 檢定結果可知，SARIMAX 模型之殘差存在顯著的變異數不恆定，顯示僅以 SARIMAX 模型描述平均結構並不足以捕捉時間序列的波動幅度變化。因此，本研究進一步對前者模型的殘差建立 GARCH(1,1) 波動模型，並假設誤差項服從 Student’s t 分配以捕捉殘差分布厚尾（fat-tail）現象，來擬合條件變異數的變化。

GARCH 模型估計結果如下：
$$
\omega = 1.36×10⁻⁶, p < 0.001
$$
$$
\alpha = 0.0501, p = 0.034
$$
$$
\beta = 0.9299, p < 0.001
$$
$$
\alpha + \beta = 0.9800 < 1
$$
其中，$\alpha$ 代表短期衝擊對波動的影響，$\beta$ 則表示波動的持續性（volatility persistence）。估計結果顯示 $\beta$ 顯著且接近 1，代表波動具有高度持續性，即當市場進入高波動狀態後，影響會維持相當長時間；而 $\alpha$ 顯著則表示新資訊衝擊確實會立即反映於條件變異數中。由於系統總慣性 $\alpha + \beta < 1$，模型滿足弱平穩條件，顯示條件變異數確實為均值回歸過程。整體而言，加入 GARCH 模型成功捕捉原先殘差中的變異數不恆定問題，相較於僅使用 SARIMAX 的模型，SARIMAX–GARCH 架構能更完整描述時間序列的統計特性與風險結構。

In [ ]:
mod_garch_t = arch_model(
    mod_sarimax.resid.dropna(),
    mean='Zero',
    vol='GARCH',
    p=1,
    q=1,
    dist='t'
)

res_vol = mod_garch_t.fit()
print(res_vol.summary())

為檢驗 GARCH 模型是否已充分捕捉條件變異數動態，本研究進一步對標準化殘差平方做 Ljung–Box test 檢定自相關性，結果顯示 p-value 為 0.983，無法拒絕「不存在自相關」的虛無假設，表示平方殘差中已無顯著序列相關，亦即波動聚集現象已被模型有效捕捉；而對剩餘 ARCH 效應檢定，Engle’s ARCH LM 檢定之 p-value 為 0.825 遠大於 0.05，顯示無法拒絕「不存在剩餘 ARCH 效應」的虛無假設，代表在 GARCH 建模後，條件異質變異問題已被充分消除。

基於以上檢定結果，推論標準化殘差平方已無顯著自相關、不存在剩餘 ARCH 效應，並且波動結構已被 GARCH 模型適當捕捉，採用 GARCH 模型在擬合條件變異數上充分且合理，成功解決原先殘差中的變異數不恆定，使整體 SARIMA–GARCH 模型架構在平均數與變異數兩個層面皆達到良好配適。

In [ ]:
grach_std_resid = res_vol.std_resid

print("LB on squared std resid")
print(acorr_ljungbox(grach_std_resid**2, lags=[24], return_df=True),"\n")

lm_stat, p_value, f_stat, f_p_value = het_arch(grach_std_resid)

print("Engle's ARCH Test (LM Test) Results")
print(f"{'Statistic Type':<25} | {'Value':<12} | {'p-value':<12}")
print("-" * 50)
print(f"{'Lagrange Multiplier (LM)':<25} | {lm_stat:<12.4f} | {p_value:<12.4f}")
print(f"{'F-statistic':<25} | {f_stat:<12.4f} | {f_p_value:<12.4f}")

經過確認，SARIMAX + GARCH 模型架構能有效的解釋原始時間序列，各項參數不僅描述統計上的動態關係，同時亦可對應至再生能源發電占比的實際能源系統運作機制。

首先，在擬合條件平均數的 SARIMAX 模型中，非季節性 AR(1) 係數為正且顯著，代表再生能源占比的變動具有高度慣性，亦即能源結構的調整屬於漸進式轉變而非短期劇烈跳動。此結果反映實際電力系統中再生能源發電佔比受裝置容量擴張、政策推進與基礎建設建置所影響，其效果往往具有持續性與累積性；非季節性 MA(1) 項呈現顯著負值，顯示當期若出現短期衝擊（如天候異常、需求波動或發電調度變化），系統會透過後續調整逐步修正，使占比回歸原有趨勢，說明再生能源占比具有一定程度的短期穩定機制。進一步觀察季節性結構，顯著的季節 AR(1) 的正係數意味著跨年度的發電型態具有延續性，例如太陽能與風力發電的季節循環氣候、日照與風場條件，會在不同年度呈現相似模式；而季節 MA(1) 項的負係數則反映季節性擾動對短期波動的修正效果，代表年度週期中若出現如極端氣候造成之偏離，其影響通常不會長期累積。
波動結構方面，GARCH 模型揭示再生能源占比的不確定性並非隨機，而具有**波動群聚（volatility clustering）**特性。ARCH 項係數 $\alpha$ 小但顯著反映近期衝擊會立即提高占比變動的不穩定程度，例如短期氣候劇變或政策訊號可能導致占比波動擴大；而 GARCH 項係數 $\beta$ 明顯高於 $\alpha$，顯示再生能源占比波動主要來自過去波動的延續效果，而非單期衝擊，意味著當系統進入高波動時期如能源轉型加速階段，不確定性往往會延續一段時間。

在此模型的基礎上，本研究最後利用其分兩階段預測出條件平均數及條件變異數，得出 2026 年 10 月之再生能源佔比之統計分布，並以占比達目標比例 0.2 做為閾值，落在分布的右尾累進機率做為達標機率。結果顯示，以模型無法拒絕之對立假設「序列服從自由度為 8.6144 之 t 分佈」計算目標門檻之達標機率，對應之臨界統計量為 $t_{critical} = 2.2777$，其右尾機率僅為 0.04%，代表在模型假設成立下，2026 年 10 月之再生能源發電佔比超過目標門檻值的**機率極低**。換言之，依據目前時間序列之趨勢結構與波動特性，在預測期內達標的可能性十分趨近於 0。統計上，此機率對應之標準化距離已超過約 2.28 個標準差，表示門檻值顯著高於模型預測之條件期望，且在考慮由 GARCH 所捕捉之波動風險後，仍難以落入可實現範圍。因此，本研究推論：若無結構性變化或外部衝擊，短期內達成該目標之可行性極低。

In [ ]:
forecast_steps = 10
sarimax_forecast = mod_sarimax.get_forecast(steps=forecast_steps, exog=np.ones((forecast_steps, 1)))
sarimax_mu_hat = sarimax_forecast.predicted_mean.iloc[-1]
garch_forecast = res_vol.forecast(horizon=forecast_steps)
garch_sigma_hat = np.sqrt(garch_forecast.variance.values[-1, -1])

threshold = 0.2

from scipy import stats
nu = res_vol.params['nu']
t_critical = stats.t.ppf(0.975, df=nu)
print(f"臨界統計值：{t_critical=:.4f}")

t_score = (threshold - sarimax_mu_hat) / garch_sigma_hat
prob_exceed = 1 - stats.t.cdf(t_score, df=nu)

print(f"基於 t 分佈 (df={nu:.4f}) 計算的達標機率：{prob_exceed:.4%}")

future_dates = pd.date_range(start='2026-01-01', periods=10, freq='MS')
mu_forecast = sarimax_forecast.predicted_mean
sigma_forecast = np.sqrt(garch_forecast.variance.values[-1, :]) 
lower_bound = mu_forecast - t_critical * sigma_forecast
upper_bound = mu_forecast + t_critical * sigma_forecast

plt.figure(figsize=(15, 7))
plt.plot(y_t.index, y_t, color='black', label='Historical Data (2006-2025)', linewidth=1.5)
plt.plot(future_dates, mu_forecast, color='blue', linestyle='--', label='Forecast Mean (2026)', linewidth=2)
plt.fill_between(future_dates, lower_bound, upper_bound, color='blue', alpha=0.2, label='95% Forecast Interval (GARCH-t)')
plt.axhline(y=threshold, color='red', linestyle=':', linewidth=2, label='Policy Target (0.2)')
plt.axvline(pd.to_datetime('2021-04-01'), color='orange', linestyle='-', alpha=0.3)
plt.text(pd.to_datetime('2021-01-01'), plt.ylim()[0], ' 2021 Policy Shift', color='orange', rotation=90, va='bottom')
plt.title('Renewable Energy Share: Historical Trend and 2026 Risk Forecast', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Share Percentage', fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.4)
plt.legend(loc='upper left')

import matplotlib.dates as mdates
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1,7]))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xticks(rotation=45)
plt.grid(True, which='major', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()